In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ase import Atoms
from samos.trajectory import Trajectory
from samos.utils.constants import *
from samos.analysis.get_gaussian_density  import get_gaussian_density 
from samos.lib.mdutils import recenter_positions

This example shows how to use a QE input file and a CP trajectory to plot the density of the Li-ions and of the rigid sublattice in LGPS.

NB: the files are not provided here, the goal is to showcase the code.

The positions of the QE input file and of the CP trajectory are aligned removing the drift of the center of the rigid sublattice.

In [11]:
def load_start_configuration_from_qe_file(file_scf_input):
     """
     Read starting postisions and nat from a QE input file
     """
     with open(file_scf_input) as finit:
          finit_lines = finit.readlines()
     iat=-1
     types=[]
     start_positions=[]
     for l in finit_lines:
          if len(l.split())>0:
               if l.split('=')[0].strip()=='nat':
                    snat=l.split('=')[1].strip()
                    if snat[-1]==',':
                         snat=snat[:-1]
                    print(snat)
                    nat=int(snat)
                    print(f'Detected {nat} atoms')
               if l.split()[0]=='ATOMIC_POSITIONS':
                    iat+=1
                    if len(l.split())>0 and l.split()[1].strip()=='bohr' or l.split()[1].strip()=='(bohr)':
                         print('Detected bohr we move to angstrom units')
                         factor=bohr_to_ang
                    else:
                         print('We assume angstrom units')
                         factor=1
               elif iat>=0 and iat<nat:
                    if not l[0]=='#':
                         split=l.split()
                         typ=split[0]
                         types.append(typ)
                         iat+=1
                         pos=np.array([float(split[1])*factor,float(split[2])*factor,float(split[3])*factor])
                         start_positions.append(pos)
     start_positions=np.array(start_positions)
     assert(len(types)==nat)
     assert(start_positions.shape==(nat,3))
     return nat, start_positions


def load_trajectory_from_cp_file(file_traj, format='bohr'):
    """
    Read trajectory from a trajectory output from cp.x
    """
    with open(file_traj) as ftraj:
        ftraj_lines = ftraj.readlines()
        nt = int(len(ftraj_lines)/(nat+1))
        positionsArr = np.zeros((nt,nat,3),dtype=float)
        for it in range(0,nt):
            every_nstep_pos = []
            for line in ftraj_lines[((nat+1)*it)+1:(nat+1)*(it+1)]:
                y = line.split()
                y = np.array(y,dtype=float)
                every_nstep_pos.append(y)
            if format=='bohr':
                positionsArr[it,:,:] = np.array(every_nstep_pos,dtype=float) * bohr_to_ang #NB: I put the positions in angstrom as required by the documentation
            else:
                positionsArr[it,:,:] = np.array(every_nstep_pos,dtype=float)
    delta=np.abs(float(ftraj_lines[0].split()[1])-float(ftraj_lines[0+nat+1].split()[1]))
    delta=delta*1000 #NB: I put the time in femtoseconds as required by the documentation --> should not matter for just getting the density
    timestep=delta 
    return positionsArr, timestep

In [5]:
### Input --> change here (and only here, hopefully)
a = 16.4294 * bohr_to_ang
b = a  
c = 16.4294 * 1.44919 * bohr_to_ang
simulation_cell=[[a,0,0],[0,b,0],[0,0,c]]
formula='Li20Ge2P4S24'
rigid_lattice=['Ge','P','S']

# from this file we take the starting positions
file_scf_input='/Users/arismarcolongo/Desktop/qe/LGPS_free8.in'
# trajectory file
file_traj='/Users/arismarcolongo/Desktop/qe/giuliana/NVE/tLGPS/NVE_850K/tLGPS_850K-NVE.pos'

In [6]:
nat, start_positions = load_start_configuration_from_qe_file(file_scf_input)
positionsArr, timestep = load_trajectory_from_cp_file(file_traj)

50
Detected 50 atoms
Detected bohr we move to angstrom units


In [7]:
## initialize atoms and trajectory object

atoms = Atoms(formula)
atoms.set_positions(start_positions)
atoms.cell = np.array(simulation_cell)
t = Trajectory()
t.set_timestep(timestep)
t.set_atoms(atoms)
t.set_positions(np.array(positionsArr))

print('We recenter so that the centre of the rigid sublattice is at zero')
t.recenter(rigid_lattice, mode='geometric')

We recenter so that the centre of the rigid sublattice is at zero


In [8]:
def evaluate_com(conf, mode=None):
    indices_rigid_z = np.array(list(t.get_indices_of_species('Ge', start=0))+\
                    list(t.get_indices_of_species('P', start=0))+\
                    list(t.get_indices_of_species('S', start=0)))
    masses = t.atoms.get_masses()
    if mode=='geometric':
        masses = [1.0]*len(masses)
    num=np.sum(np.array([conf[i,:]*masses[i] for i in indices_rigid_z]), axis=0)
    den=np.sum(np.array([masses[i] for i in indices_rigid_z]))
    return num/den
    
com = evaluate_com(start_positions, mode='geometric')

In [9]:
# we shift all again so that the com of the rigid sublattice is aligned with the centre at the starting configuration
shift_all=True
if shift_all:
    pos=t.get_positions()
    nstep, nat, ncoord = pos.shape
    for i in range(nstep):
        for j in range(nat):
            pos[i,j,:]=pos[i,j,:]+com #how to do better with broadcast ?
    #probably not needed (pos is a reference) but but just to be sure
    t.set_positions(np.array(pos))

In [10]:
indices_li = t.get_indices_of_species('Li', start=1)
indices_rigid = np.array(list(t.get_indices_of_species('Ge', start=1))+\
                 list(t.get_indices_of_species('P', start=1))+\
                 list(t.get_indices_of_species('S', start=1)))

get_gaussian_density(t, element=None, 
                     outputfile='out_li_all_test_2.xsf',
                     indices_i_care=indices_li, 
                     indices_exclude_from_plot=[]) 

get_gaussian_density(t, element=None, 
                     outputfile='out_rigid_all_test_2.xsf',
                     indices_i_care=indices_rigid, 
                     indices_exclude_from_plot=[]) 

(get_gaussian_density) indices_i_care: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
Grid is 87 x 87 x 126
Box is  8.694064069089048 x 8.694064069089048 x 12.59935070828316
Writing xsf file to out_li_all_test_2.xsf
(get_gaussian_density) We do not show these atoms in the xsf file: []
=(get_gaussian_density) indices_i_care: [21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44
 45 46 47 48 49 50]
===============================================================================Grid is 87 x 87 x 126
Box is  8.694064069089048 x 8.694064069089048 x 12.59935070828316
Writing xsf file to out_rigid_all_test_2.xsf
(get_gaussian_density) We do not show these atoms in the xsf file: []